In [ ]:
from nilearn import plotting
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import scipy.io as sio
import networkx as nx

In [ ]:
colors_networks = np.array([
    "#FF0000",  # Red DMN
    "#00FF00",  # Green VIS
    "#0000FF",  # Blue FP
    "#FFFF00",  # Yellow Reward
    "#FF00FF",  # Magenta DA
    "#000000",  # nan
    "#00FFFF",  # Cyan VA
    "#800080",  # Purple SN
    "#FFA500",  # Orange CO
    "#008000",  # Dark Green SM Dorsal
    "#800000",  # Maroon SM Lateral
    "#008080",  # Teal AUD
    "#000000",  # nan
    "#000000",  # nan
    "#000080",  # Navy ParietoMedial
    "#000000",  # nan
    "#FFD700",  # Gold Medial Temporal Lobe
    "#000000",  # nan
])

In [ ]:
coordinates = np.loadtxt("/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/func_networks/masks/ROIs_300inVol_MNI/coords_300.csv", delimiter=",", dtype=float)
func_network_affiliation = np.loadtxt("/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/func_networks/masks/ROIs_300inVol_MNI/ROIs_CommunityAffiliation.txt").astype(int)

colors = np.zeros((300, 4))
z_min, z_max = coordinates[:,2].min(), coordinates[:,2].max()
for i, (x, y, z) in enumerate(coordinates):
    alpha = (z+52.0)/(52.0+74.98)
    hex_color = colors_networks[func_network_affiliation[i]-1].lstrip("#")
    r,g,b = tuple(int(hex_color[i:i+2], 16)/255 for i in (0, 2, 4))
    colors[i, :] = [r,g,b,alpha]

In [ ]:
#fig = plt.figure(figsize=(10, 10))
plotting.plot_connectome(
    np.zeros((300, 300)),
    coordinates,
    edge_threshold="95%",
    node_color="#888888",
    node_size=24,
    title=f"",
    display_mode="z",
    annotate=False,
    edge_kwargs={"linewidth": 100},
    #figure=fig
)

plt.savefig("figuras/roi300_negro.eps")

In [ ]:
np.random.seed(40)
y = np.random.rand(350)
y_smooth = savgol_filter(y, 20, 2)
fig, ax = plt.subplots(1, figsize=(20, 5))
ax.plot(y_smooth, linewidth=15, c="black")
ax.set_ylim([0, 1])
plt.axis('off')

plt.savefig("figuras/timeseries.png")

In [ ]:
MAIN_DIR = "/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn"

connectivity_regions = sio.loadmat(f"{MAIN_DIR}/results/connectivity/300/connectivity_300.mat")["C"]

c = connectivity_regions[1,1,:,:]

fig, ax = plt.subplots(1, figsize=(8,8))
ax.imshow(c, cmap="turbo")
plt.axis("off")

plt.savefig("matriz.png")

In [ ]:
MAIN_DIR = "/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn"

connectivity_regions = sio.loadmat(f"{MAIN_DIR}/results/connectivity/300/connectivity_300.mat")["C"]

func_network_affiliation_clean = func_network_affiliation[12:].copy()
func_network_affiliation_clean[140:] = 42

c = connectivity_regions[3,2,12:,12:].copy()
rois_in_network = func_network_affiliation_clean == 1
c[~rois_in_network, :] = -10
c[:, ~rois_in_network] = -10

fig, ax = plt.subplots(1, figsize=(8,8))
ax.imshow(c, cmap="turbo", vmin=-1, vmax=1)
plt.axis("off")

plt.savefig("figuras/matriz-intra.png")

In [ ]:
c = connectivity_regions[3,2,12:,12:]
func_network_affiliation_clean = func_network_affiliation[12:]
a = func_network_affiliation_clean == 10
index = np.where(a)[0]

plt.imshow(c[index,:][:,index])

In [ ]:
MAIN_DIR = "/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn"

connectivity_regions = sio.loadmat(f"{MAIN_DIR}/results/connectivity/300/connectivity_300.mat")["C"]

c = connectivity_regions[3,2,12:,12:]

z = np.ones((288, 288))*(-10)
for i in range(288):
    for j in range(288):
        if func_network_affiliation[i] == 1 and func_network_affiliation[j] != 1:
            z[i,j] = c[i,j]
        if func_network_affiliation[i] != 1 and func_network_affiliation[j] == 1:
            z[i,j] = c[i,j]    

fig, ax = plt.subplots(1, figsize=(8,8))
ax.imshow(z, cmap="turbo", vmin=-1, vmax=1)
plt.axis("off")

plt.savefig("figuras/matriz-inter.png")

In [ ]:
MAIN_DIR = "/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn"

connectivity_regions = sio.loadmat(f"{MAIN_DIR}/results/connectivity/300/connectivity_300.mat")["C"]

c = connectivity_regions[3,2,12:,12:]

fig, ax = plt.subplots(1, figsize=(8,8))
ax.imshow(c, cmap="gray", vmin=-1, vmax=1)
plt.axis("off")

plt.savefig("figuras/matriz-gris.png")

In [ ]:
import matplotlib.cm as cm
import matplotlib

norm = matplotlib.colors.Normalize(vmin=0, vmax=1.3, clip=True)
mapper = cm.ScalarMappable(norm=norm, cmap=cm.plasma)

fig, ax = plt.subplots(1, figsize=(8,8))
x = np.arange(0, 1, 0.01)
ax.scatter(x, -(1.5*x-0.75)**2 + 0.75, s=100, c=list(map(lambda x: mapper.to_rgba(x), x)))
ax.spines[['right', 'top']].set_visible(False)
plt.setp(ax.spines.values(), linewidth=5)
ax.set_ylim([0, 1])
ax.set_xticks([0], "")
ax.set_yticks([0], "")

plt.rcParams.update({'font.size': 30})
plt.tight_layout()
plt.savefig("figuras/medialuna.png")